## 시계열 데이터 (Sequence Data) 의 소개
- 사전적 의미 : " 일정 시간 간격으로 배치된 데이터들의 수열을 말한다 " 라고 위키피디아가 말한다.
- 실용적 포인트 : 정보의 **배치 순서**가 **의미**가 있을때!!

### 예시
1. 주식가격
2. 버스 도착시간
3. 음악
4. 언어

### Q: 시계열 데이터는 어떻게 수치화 하는가?
인간은 큰 노력 없이 주어지는 정보를 받아드린다. 시각, 청각, 촉각 등등 그 자체로 뇌는 학습한다(물론 뉴런들 간의 신경전달물질 이동 ...). 하지만 컴퓨터는 정보를 숫자로만 받아드린다. 고로, 프로그램으로 문제를 해결할 때 가장 중요한 Task 중 하나가 주어진 데이터를 수치화 하는것이다!!

우리들은 직접 위의 예시 데이터들을 만져보도록 하겠다

## Example 1) 주식 가격
주식이라는 단어를 들었을때 생각을 해보자.

- 주식은 **시간**에 따라 **가격**이 변동한다. 2가지의 variable 이 존재함을 인지하자.
- 시간을 discrete 하게 나타내기 위해 sample 주기가 필요함을 인지하자.
- 가격은 화폐단위를 고려하고, 수학적 모델링을 위한 가공 방법을 고민해보자.

이제 본격적으로 실시간 주식 데이터와 놀아보자

In [ ]:
# 수업자료 깃허브 Repository를 다운받아준다
!git clone https://github.com/HanyangTechAI/NLP_2020.git .

In [1]:
# 필요한 패키지 (인생을 편하게 해주는 미리 만들어놓은 프로그램이라고 보면 된다) 를 설치시킨다
!cat ./lab01_all_about_sequence/requirements.txt
!echo '----------------------------------'
!pip install -r ./lab01_all_about_sequence/requirements.txt

yfinance
numpy
pandas
matplotlib
torch
torchvision
----------------------------------


In [2]:
# 주식 Ticker Symbol (시세 기호) 를 Dictionary 형태로 모아봅시다
tickers = {
    "apple" : "AAPL",
    "google" : "GOOGL",
    "amazon" : "AMZN"
}

print(tickers["apple"])

AAPL


In [3]:
# yfinance 패키지를 import 시켜준다
import yfinance as yf

# yfinance 가 어떻게 작동하는지 예시를 돌려본다

## Argument 첫번째: ticker symbol / 두번째: 데이터 요청 시작 날자 / 세번째: 데이터 요청 끝 날자
## 반환값은 pandas.Dataframe 형임을 알수있다
data = yf.download(tickers["apple"],'2019-01-01','2020-01-01')
data.head()  # 첫 5개의 row 만 출력해준다

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-12-31,158.529999,159.360001,156.479996,157.740005,155.405045,35003500
2019-01-02,154.889999,158.850006,154.229996,157.919998,155.582367,37039700
2019-01-03,143.979996,145.720001,142.000000,142.190002,140.085220,91312200
2019-01-04,144.529999,148.550003,143.800003,148.259995,146.065353,58607100
2019-01-07,148.699997,148.830002,145.899994,147.929993,145.740265,54777800


In [ ]:
# data를 2차원 행렬로 표현할때 shape을 출력해본다

print('shape: {}'.format(data.values.shape))

In [ ]:
# 위 출력 결과를 보면, 1일 당 6개의 column이 존재함을 알 수 있다
# 이중 'Adj Close' column 에 대해 그래프를 그려보겠다

# 먼저 matplotlib 이라는 visualization 라이브러리를 import 하겠다
import matplotlib.pyplot as plt
%matplotlib inline

data['Adj Close'].plot()
plt.show()

In [ ]:
# 이제 데이터를 수치적으로 분석을 해보자
# 먼저 수치적 데이터를 다루는 numpy 라이브러리를 import 해준다
import numpy as np

# 'Adj Close' column에 대해 분석을 해보자
temp_data = data['Adj Close'].values

# Minimum value and its index
min_val = np.min(temp_data)
min_index = np.argmin(temp_data, axis=0)
print('min: {}\nmin_index: {}\n'.format(min_val, min_index))

# Maximum value and its index
max_val = np.max(temp_data)
max_index = np.argmax(temp_data, axis=0)
print('max: {}\nmax_index: {}\n'.format(max_val, max_index))

# Mean
mean = np.mean(temp_data)
print('mean: {}\n'.format(mean))

#Variance
variance = np.var(temp_data)
print('variance: {}'.format(variance))

## Example 2) 버스 도착 시간
도착 시간은 어떤 특성을 가질까?

- 변수: **장소**, **버스 id**
- **장소**와 **버스 id**가 fix 되면 버스의 도착 시간이 mapping 될거다.

이러한 점들을 생각을 하고 데이터를 모아보자

### 공공 데이터 포털
정부에서는 '공공 데이터 포털'을 운영하며 기업이나 민간인들이 자유롭게 정보를 접할 수 있도록 API를 제공한다. 우리들도 '버스도착정보 조회 서비스' 를 이용해 데이터를 모아보겠다.

링크: https://www.data.go.kr/subMain.jsp?param=T1BFTkFQSUB1bmRlZmluZWQ=#/L3B1YnIvcG90L215cC9Jcm9zTXlQYWdlL29wZW5EZXZHdWlkZVBhZ2UkQF4wMTJtMSRAXnB1YmxpY0RhdGFQaz11bmRlZmluZWQkQF5wdWJsaWNEYXRhRGV0YWlsUGs9dWRkaTo5OWQ5OGM3YS1jNWNiLTQ2YjktYWZiZS1hMTBhODA0OTc2ZGEkQF5vcHJ0aW5TZXFObz0xNDQ5JEBebWFpbkZsYWc9dHJ1ZQ==

In [ ]:
import requests  # url 요청시 사용
from bs4 import BeautifulSoup as bs  # xml parsing 시 사용

In [ ]:
# 요청할 url 을 구성해보자.
# 위 링크의 documentation 을 참고한다.

skeleton_url = 'http://ws.bus.go.kr/api/rest/arrive/getArrInfoByRouteAll?{query}'

# query string 을 만든다
# 버스도착시간 서비스의 경우 ServiceKey(서비스 요청 권한 ID)와 busRouteId(노선 ID)를 필요로 한다
serviceKey = 'VssaYyzg2H6zSuKOVXfD2wpUZm2UszZJEakrwaG7K1vpjdUo1mSCzgdF7ODLGqSHKA1FF9o2Xz1sJ8NXe0nHpg%3D%3D'
busRouteId = '100100118'
query = 'ServiceKey={serviceKey}&busRouteId={busRouteId}'.format(serviceKey=serviceKey, busRouteId=busRouteId)

# query와 skeleton_url을 붙여준다
request_url = skeleton_url.format(query=query)

print(request_url)

In [ ]:
# request_url로 request를 날린다
req = requests.get(request_url)  # req로 request를 받는다
html = req.text
print(html)

In [ ]:
# BeautifulSoup의 html parser를 이용해 응답받은 내용을 parsing 한다
soup = bs(html, 'html.parser')

In [ ]:
print(soup.prettify())

In [ ]:
# 100100118 루트가 다니는 모든 정류소 명을 찾아보자
bus_stations = soup.find_all('stnm')
bus_stations = list(map(lambda elem: elem.text, bus_stations))
print(bus_stations)

In [ ]:
data = []

items = soup.find_all('itemlist')
for item in items:  # Iter through all items
    temp_datum = {}
    
    children = item.findChildren()
    for child in children:
        tag_name = child.name
        value = child.text
        
        temp_datum[tag_name] = value
    
    data.append(temp_datum)

In [ ]:
import pandas as pd

dt = pd.DataFrame(data)
dt.head()

## Example 3) 음악
음성은 어떠한 특성을 가질까? 유튜브를 통해 음악 데이터를 분석해보자

In [ ]:
!pip install youtube_dl
!pip install pydub
!apt install ffmpeg

In [ ]:
# 유튜브 링크로부터 음성 파일(mp3)을 다운받습니다.

import youtube_dl

link = 'https://www.youtube.com/watch?v=HZcuSplFrS4'  # Youtube 링크

def download(link):
    ydl_opts = {
      'format': 'bestaudio/best',
      'postprocessors': [{
          'key': 'FFmpegExtractAudio',
          'preferredcodec': 'mp3',
          'preferredquality': '192',
      }],
    }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
        result = ydl.extract_info(link,download=False)
    if 'entries' in result:
        video = result['entries'][0]
        return 0
    else:
        video = result
        title = "{}-{}.mp3".format(video["title"],video['id'])
        ydl.download([link])
        return title
      
file_name = download(link)
print(file_name)

In [ ]:
# mp3 파일을 wav 파일로 converting 후
# 음성 파형 데이터를 반환합니다
import tempfile
import os
import pydub
import scipy
import scipy.io.wavfile

def read_mp3(file_path, as_float = True):
    path, ext = os.path.splitext(file_path)
    assert ext=='.mp3'
    mp3 = pydub.AudioSegment.from_mp3(file_path)
    _, path = tempfile.mkstemp()
    mp3.export(path, format="wav")
    rate, data = scipy.io.wavfile.read(path)
    os.remove(path)
    if as_float:
        data = data/(2**15)
    return rate, data

file_path = './{}'.format(file_name)

rate, data = read_mp3(file_path)
data = data[:, 0]  # stereo 환경에서 왼쪽 데이터만

In [ ]:
print('rate: {}'.format(rate))
print('data shape: {}'.format(data.shape))

In [ ]:
# plotting 을 해보자
%matplotlib inline
import matplotlib
import numpy as np
import matplotlib.pyplot as plt

start_t_sec = 10  # 시작 시간 (초)
end_t_sec = 11# 종료 시간 (초)

start_tick = int(start_t_sec*rate)  # 시작 in tick
end_tick = int(end_t_sec*rate)  # 종료 in tick

temp_data = data[start_tick: end_tick]

plt.plot(temp_data)
plt.ylabel('magnitude')
plt.xlabel('time step')
plt.show()